In [52]:
import copy

list1 = [1, 2, 4]

list2 = copy.deepcopy(list1)

print(list2)

[1, 2, 4]


In [81]:
import random

# 初始化牌局，洗牌
game = game()

# 初始化两个选手
player1 = player(game.fapai())
player2 = player(game.fapai())

# 摸牌的顺序，游戏没完成就循环执行, 要么有人胡，要么牌摸完了
turn = random.random() >= 0.5
while not game.finished:
    if turn: # 玩家 1 出牌
        pai = player1.chupai_process(game)
        if game.finished:
            player2.score = -player1.score
        else:
            player2.oppo_pai = pai
    else: # 玩家 2 出牌
        pai = player2.chupai_process(game)
        if game.finished:
            player1.score = -player2.score
        else: 
            player1.oppo_pai = pai
    turn = !turn
    

print("player1: data", player1.data, " score: ", player1.score)
print("player2: data", player2.data, " score: ", player2.score)

KeyError: 12

In [87]:
import collections

class game:
    def __init__(self):
        # 一共有多少种类型的牌
        self.pai_types = {'WAN_1':1 , 'WAN_2':2, 'WAN_3':3, 'WAN_4':4,'WAN_5':5,'WAN_6':6, 'WAN_7':7, 
                          'WAN_8':8, 'WAN_9':9, 'DONGFENG':10, 'NANFENG':11, 'XIFENG':12, 'BEIFENG':13, 
                          'BAIBAN':14, 'FACAI':15,  'HONGZHONG':16}
        self.pais = self.gengerate_pais() # 生成一副牌
        self.finished = False # 结束，有人胡或没有牌了
        self.pais_finished = False # 没有牌了结束

    def gengerate_pais(self):
        """
        生成一副牌
        """
        pais = [v for v in  list(self.pai_types.values()) for i in range (0, 4)]
        random.shuffle(pais)
        return pais
    
    def mopai(self):
        """
        摸牌
        """
        return self.pais.pop()
 
    def fapai(self):
        """
        首次发牌，一共 13 张
        """
        return [self.pais.pop() for i in range(0, 13)]
    
class player:
    def __init__(self, my_pais):
        self.new_pai =  None    # 我新进的最新一张牌
        self.zimo = False #  True 新牌是自己摸的牌 False:碰、胡、杠、吃的对家的牌
        
        # 我的所有牌的组成
        self.dynamic_pais = my_pais  # 手上的牌
        # self.static_pais = {}   # 碰、杠、吃的牌
        self.oppo_pai = None    # 对手出的牌
        self.last_pai = False   # 对手是否是出的最后一张牌
        self.output_pais = []   # 我出过的牌
        self.ting  = False      # 是否听牌
        self.data = { # 当前胡牌组成的序列，把从对家碰、杠、吃的牌放里边
            'sunzi': {},
            'kezi': {},
            'gang': {},
            'jiang': {},
            'pais': [],
            'lastpai':None
        }
        self.score = 0 # 当前胡牌得到的分数
        

    def chupai_process(self, game):
        """
        能胡就胡能碰就碰能杠就杠能吃就吃，这个地方到时可以换成深度学习动作
        
        game：表示当前游戏
        """
        
        # 使用对家出的牌胡、碰、杠，不是自己摸的
        self.new_pai = self.oppo_pai
        self.zimo = False
        
        if self.oppo_pai == None: # 对家还没有出牌（第一个出牌的），直接摸牌
            self.mopai(game)
        elif self.is_hu(): # 胡
            game.finished = True
            return
        elif self.is_gang(): # 杠
            self.gang()
            self.mopai(game) # 杠完之后还要摸牌
        elif self.is_peng(): # 碰
            self.peng()
        elif is_chi(): # 吃
            self.chi()
        else: # 摸牌
            self.mopai(game)
        
        if not game.finished: # 不是胡牌或流局就必须出牌
            
            pai = self.chu_pai()
            self.output_pais.append(pai)
            
            return pai
        
        return None
            
    def mopai(self, game):
        """
        摸牌，注意：先判断还有没有牌，若没有牌，牌局结束，流局
        """
        
        if len(game.pais) == 0: # 流局、牌局结束
            game.pais_finished = True
            game.finished = True
            return
        else:
            pai = game.mopai() # 摸牌
            self.new_pai = pai
            self.zimo = True # 表示是自己摸的牌
            if self.is_hu(): # 能胡就胡
                game.finished = True
                return
            elif self.is_gang(): # 能杠就杠，杠完再摸牌
                self.mopai(game)
            else: # 否则摸牌
                self.dynamic_pais.append(pai)
    
    def is_peng(self):
        """
        判断是否可以碰对家出的牌
        """
        return self.dynamic_pais.count(self.oppo_pai) >= 2

    def peng(self):
        """
        碰牌且把相关牌从动态牌面中移除，添加到静态牌面
        """
        removeEle(self.dynamic_pais, self.oppo_pai, 2)
        self.data['kezi'][self.oppo_pai] = {'times':1, 'zimo': 0}

    def is_chi(self):
        """
        判断是否可以吃对家出的牌
        """
        
        if self.oppo_pai <= 9 and pai - 2 in self.dynamic_pais and pai - 1 in self.dynamic_pais:
            return True
        elif self.oppo_pai <= 8 and pai - 1 in self.dynamic_pais and pai + 1 in self.dynamic_pais:
            return True
        elif self.oppo_pai <= 7 and pai + 1 in self.dynamic_pais and pai + 2 in self.dynamic_pais:
            return True
        return False

    def chi(self):
        """
        吃牌且把相关牌从动态牌面中移除，添加到静态牌面
        """
        
        if self.oppo_pai <= 9 and pai - 2 in self.dynamic_pais and pai - 1 in self.dynamic_pais:
            self.data['sunzi'][pai - 2] = {'times': self.get_times_by_type(pai - 2) + 1, 'zimo': False}
            self.dynamic_pais.remove(pai - 1)
            self.dynamic_pais.remove(pai - 2)
        elif self.oppo_pai <= 8 and pai - 1 in self.dynamic_pais and pai + 1 in self.dynamic_pais:
            self.data['sunzi'][pai - 1] = {'times': self.get_times_by_type(pai - 1) + 1, 'zimo': False}
            self.dynamic_pais.remove(pai - 1)
            self.dynamic_pais.remove(pai + 1)
        elif self.oppo_pai <= 7 and pai + 1 in self.dynamic_pais and pai + 2 in self.dynamic_pais: 
            self.data['sunzi'][pai] = {'times': self.get_times_by_type(pai) + 1, 'zimo': False}
            self.dynamic_pais.remove(pai + 1)
            self.dynamic_pais.remove(pai + 2)
    
    def is_gang(self):
        """
        判断是否可以杠，如果自己摸的牌在静态牌和动态牌中都需要判断一下（暗杠），如果是杠的对家的牌，只在动态牌中查找（明杠）
        """
        if self.zimo:
            if self.data['kezi'].get(self.new_pai) != None: # 有碰牌
                return True
            
        return self.dynamic_pais.count(self.new_pai) == 3

    def gang(self):
        """
        杠，包括明杠和暗杠
        """
        
        if self.zimo:
            self.data['gang'][self.new_pai] = {'times':1, 'zimo': True}
            
            if self.data['kezi'].get(self.new_pai) != None: # 有刻子，从刻子移除，转到杠
                del self.data['kezi'][self.new_pai]
            else:
                removeEle(self.dynamic_pais, self.new_pai, 3)
        else:
            self.data['gang'][self.new_pai] = {'times':1, 'zimo': False} 
            removeEle(self.dynamic_pais, self.new_pai, 3)    
            
    def get_times_by_type(self, pai_type, pai):
        """
        获取指定的牌组合出现的次数
        
        pai_type: 牌组合的类型
        pai: 牌编号
        """
        times = 0
        result = self.data[pai_type].get(pai)
        if result != None:
            times = result['times']
        return times    
            
    def removeEle(self, pais, pai, num):
        """
        删除指定元素
        
        pais: 需要删除元素的列表
        pai : 需要删除指定的元素
        num : 删除几个
        """
        count = 0
        for index, p in enumerate(pais):
            if p == pai and count < num:
                del pais[index]
                count += 1

    import copy

    def is_hu(self):
        """
        判断是否可以胡牌，牌可以是自己摸的牌或胡对家的牌
        计算每一种胡牌的可能的排列组合
        """

        # 加上 new_pai 判断,计算每个动态牌出现的次数
        dynamic_pais = copy.deepcopy(self.dynamic_pais)
        dynamic_pais.append(self.new_pai)
        pai_count = {pai: 0 for pai in range(1, 17)}
        for pai in dynamic_pais:
            pai_count[pai] += 1

        # 所有可能的对子作为将, 计算每种胡的可能
        score = 0
        d = None
        
        for i in pai_count.keys():
            if pai_count[i] >= 2: # 大于 2 表示可以作为将

                pai_count[i] -= 2
                
                # data 进行深度复制
                data = copy.deepcopy(self.data)
                
                # 判断是不是已经用过了对家出的牌，对家牌只能使用一次
                used_oppo_pai = False
                zimo = True
                if not self.zimo and self.new_pai == i:
                    used_oppo_pai = True
                    zimo = False
                
                result = []
                data['jiang'][i] = {'times': self.get_times_by_type('jiang', i) + 1, 'zimo': zimo}
                self.check_sequence_triplet(pai_count, data, result, used_oppo_pai)
                
                for d in result:
                    # 为 True 表示能胡牌,计算番数
                    total_score = cal_score(d)
                    if total_score > score:
                        score = total_score
                        d = data
                
                pai_count[i] += 2
        
        # 有分数表示能胡牌
        if score > 0:
            self.score = score
            return True
        else:
            return False
    
    def check_sequence_triplet(self, pai, data, result, used_oppo_pai):
        """
        需要先确定静态牌，去掉静态牌后的牌能组成的牌，全是对子的还没有包括
        
        pai_count:每个牌出现的次数
        data：碰、杠、吃、将的序列
        """ 

        # 判断是否所有元素已经使用，是：表明已经递归到结束(注意：只有在 6 对或没有将牌的时候才算胡)
        if len(list(filter(lambda x: x != 0, pai.values()))) == 0:
            jiang_len = len(list(data['jiang'].keys()))
            if jiang_len == 6 or jiang_len == 0:
                result.append(data)
                return
            else:
                return

        # i 表示牌的编号
        for i in pai.keys():
            # 判断有没有刻子
            if pai[i] >= 3:
                pai[i] -= 3
                
                data_kezi = copy.deepcopy(data)
                
                zimo = True
                used_oppo = copy.deepcopy(used_oppo_pai)
                if not used_oppo and not self.zimo and i == self.new_pai:# 判断这张牌是不是用的对家出的牌
                    used_oppo = True
                    zimo = False
                
                data_kezi['kezi'][i] = {'times': self.get_times_by_type('kezi', i) + 1, 'zimo': zimo}
                self.check_sequence_triplet(pai, data_kezi, result, used_oppo)
                pai[i] += 3
            
            # 判断有没有顺子
            sunzi = i + 2 <= 9 and pai[i] >= 1 and pai[i + 1] >= 1 and pai[i + 2] >= 1
            if (sunzi):
                pai[i]     -= 1
                pai[i + 1] -= 1
                pai[i + 2] -= 1
                
                data_sunzi = copy.deepcopy(data)
                
                zimo = True
                used_oppo = copy.deepcopy(used_oppo_pai)
                if not used_oppo_pai and not self.zimo and i == self.new_pai:# 判断这张牌是不是自己用的对家出的牌
                    used_oppo = True
                    zimo = False
                
                data_sunzi['sunzi'][i] = {'times': self.get_times_by_type('sunzi', i) + 1, 'zimo': zimo}
                self.check_sequence_triplet(pai, data_sunzi, result, used_oppo)
                
                pai[i]     += 1
                pai[i + 1] += 1
                pai[i + 2] += 1

            # 判断是不是对子
            if pai[i] >= 2:
                pai[i] -= 2
                
                data_duizi = copy.deepcopy(data)
                
                zimo = True
                used_oppo = copy.deepcopy(used_oppo_pai)
                if not used_oppo_pai and not self.zimo and i == self.new_pai:# 判断这张牌是不是自己用的对家出的牌
                    used_oppo = True
                    zimo = False
                
                data_duizi['jiang'][i] = {'times': self.get_times_by_type('jiang', i) + 1, 'zimo': zimo}
                self.check_sequence_triplet(pai, data_duizi, result, used_oppo)
                
                pai[i] += 2
        
        return False


    def chu_pai(self):
        """
        TODO
        出牌逻辑，这部分待优化，出单牌，没有单牌出第一张牌
        """
        
        pai = self.dynamic_pais[0] # 没有单牌，默认出第一个牌
        
        pai_count = collections.Counter(self.dynamic_pais)
        for (k, v) in pai_count.items():
            if v == 1:
                pai = k
                break
        
        self.dynamic_pais.remove(pai)
        
        return pai

    def cal_score(self, pais):
        """
        计算牌的得分
        """
        pass

SyntaxError: can't assign to function call (<ipython-input-87-de0cceeb2d12>, line 332)

In [220]:
data =  {
    'gang':{# 0 表示明杠，1 表示暗杠
        14:{
            'times':1,
            'zimo': 1# zimo 表示是否是自己摸的
        }
    },
    'kezi':{
        10: {
            'times':1,
            'zimo': 1,# zimo 表示是否是自己摸的
        },
    },
    'sunzi':{
         4: { # 4 表示　456
             'times': 1,# times 表示次数
             'zimo': 1,# zimo 表示是否是自己摸的
         },
    },
    'jiang':{
        1: {
           'times': 2,# times 表示次数
           'zimo': 1,# zimo 表示是否是自己摸的
        },
        2: {
           'times': 1,# times 表示次数
           'zimo': 1,# zimo 表示是否是自己摸的
        },
    },
    'oppopai':1, # 对手最后出的牌
    'pais':[1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9] # 我的手上的所有牌
}


def count_pais_by_types_and_range(data = {}, types = [], bein = 0, end = 0):
    """
    计算指定类型的牌组合（比如刻子、杠等）包含指定范围牌的数量
    
    刻子（杠）、顺子、将，不能混用

    pais ：胡牌的所有牌类型的组合数据
    types：牌的类型
    bein :开始牌
    end  ：结束牌（包含）
    """
    result = []
    for t in types:
        for k, v in data[t].items():
            result.extend([k] * v['times'])
     
    return len(list(filter(lambda x: x in range(bein, end + 1), result)))



is_dasixi(data)

False

In [210]:
import numpy as np
import collections


def is_dasixi(data):
    """
    大四喜，4 副风刻（杠）加一对将牌组成的牌型
    """
    return count_pais_by_types_and_range(data, ['gang', 'kezi'], 10, 13) == 4

def is_dasanyuan(data):
    """
    大三元，牌里有中、发、白 3 副刻子(杠)
    """
    return count_pais_by_types_and_range(data, ['gang', 'kezi'], 14, 16) == 3
    
def is_jiubaoliandeng(data):
    """
    九宝莲灯，由一种花色序数牌子按1112345678999组成的特定牌型，见同花色任何 1 张序数牌即成胡牌
    """
    
    # 我手上的所有牌，进行升序排列
    pais = data['pais']
    pais.sort()
    
    # 对手出牌
    oppopai = data['oppopai']   
    
    return pais == [1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9] and oppopai <= 9

def is_shibaluoghan(data):
    """
    十八罗汉，牌里有 4 副杠，明暗杠均可
    """
    return count_pais_by_types_and_range(data, ['gang'], 1, 16) == 4

def is_qixinlianzhu(data):
    """
    七星连珠，由一种花色序数牌且序数相连的 7 个对子组成的牌型
    """
    result = list(data['jiang'].keys())
    result.sort()
    return result in [[1, 2, 3, 4, 5, 6, 7],\
                      [2, 3, 4, 5, 6, 7, 8],\
                      [3, 4, 5, 6, 7, 8, 9]]

def is_beidouqixin(data):
    """
    北斗七星，由 7 对字牌组成
    """
    return count_pais_by_types_and_range(data, ['jiang'], 10, 16) == 7

def is_xiaosixi(data):
    """
    小四喜，牌里有风牌的 3 副刻子及将牌
    """
    
    feng_kezi_count = count_pais_by_types_and_range(data, ['kezi', 'gang'], 10, 13)
    feng_jiang_count = count_pais_by_types_and_range(data, ['jiang'], 10, 13)
    return feng_kezi_count == 3 and feng_jiang_count == 1
    
def is_xiaosanyuan(data):
    """
    小三元，牌里有箭牌的两副刻子及将牌
    """
    
    jian_kezi_count = count_pais_by_types_and_range(data, ['kezi', 'gang'], 14, 16)
    jian_jiang_count = count_pais_by_types_and_range(data, ['jiang'], 14, 16)
    return jian_kezi_count == 2 and jian_jiang_count == 1
    
def is_guoxuedashi(data):
    """
    国学大师，牌型由字牌的刻子（杠）、将组成
    """
    
    zi_kezi_count = count_pais_by_types_and_range(data, ['kezi', 'gang'], 10, 16)
    zi_jiang_count = count_pais_by_types_and_range(data, ['jiang'], 10, 16)
    return zi_kezi_count == 4 and zi_kezi_count == 1
    
def is_sianke(data):
    """
    四暗刻，牌里有4个暗刻（暗杠）
    """
    
    count = 0
    for k, v in data['gang'].items():
        if v['zimo'] == 1:
            count += 1;
    return count == 4
    
"""
天胡、地胡
"""

def is_sangang(data):
    """
    三杠，牌里有 3 副杠，明暗杠均可
    """
    
    return count_pais_by_types_and_range(data, ['gang'], 1, 16) == 3
    
def is_hunyaojiu(data):
    """
    混幺九，由字牌和序数牌一、九的刻子及将牌组成的牌型
    """
   
    for pai in list(data['gang'].keys()) + list(data['kezi'].keys()) + list(data['jiang'].keys()):
        if pai not in [1, 9, 10, 11, 12, 13, 14, 15, 16]:
            return False
    
    for pai in data['sunzi'].keys():
        if pai not in [1, 7]:
            return False    
    
    return True

def is_qidui(data):
    """
    七对，牌型由 7 个对子组成
    """
    return count_pais_by_types_and_range(data, ['jiang'], 1, 16) == 7

def is_qingyise(data):
    """
    清一色，牌型由一种花色的序数牌组成
    """
    
    for pai in data['pais'] + [data['oppopai']]:
        if pai > 9:
            return False
    
    return True
    
def is_qinglong(data):
    """
    清龙，有 123，456，789 三付顺子即可
    """
    
    for pai in [1, 4, 7]:
        if pai not in data['sunzi'].keys():
            return False
    
    return True

def is_dayu5(data):
    """
    大于 5 牌型由序数牌 6-9 的顺子、刻子(杠)、将牌组成
    """
    
    for pai in data['pais'] + [data['oppopai']]:
        if pai <= 5 or pai > 9:
            return False
    
    return True

def is_xiaoyu5(data):
    """
    小于 5 牌型由序数牌 1-4 的顺子、刻子、将牌组成
    """
    
    for pai in data['pais'] + [data['oppopai']]:
        if pai >= 5:
            return False
    
    return True

def is_sanfengke(data):
    """
    三风刻, 牌里有3个风刻（杠）
    """
    return count_pais_by_types_and_range(data, ['kezi', 'gang'], 10, 13) == 3

def is_haidilaoyue(pais):
    """
    海底捞月，自摸或胡对方打出的最后一张牌
    TODO
    """
    pass

def is_gangshangkaihua(pais):
    """
    杠上开花，开杠抓进的牌成胡牌（不包括补花）
    TODO
    """
    pass

def is_pengpenghu(pais):
    """
    碰碰胡，牌型由 4 副刻子（或杠）、将牌组成
    """
    return count_pais_by_types_and_range(data, ['kezi', 'gang'], 1, 16) == 4

def is_hunyise(data):
    """
    混一色，牌型万字牌及字牌组成
    """
    
    zi_pai, wan_pai = False, False
    for pai in data['pais'] + [data['oppopai']]:
        if pai > 9:
            zi_pai = True
        if pai <= 9:    
            wan_pai = True
    
    return zi_pai and wan_pai

def is_quanqiuren(data):
    """
    全求人，胡牌时，全靠吃牌、碰牌、单钓别人打出的牌胡牌
    """
    pass

def is_quanangang(data):
    """
    全暗杠，牌里至少两幅暗杠
    """
    
    count = 0
    for pai in data['gang'].values():
        if pai['zimo'] == 1:
            count += 1
    
    return count >= 2

def is_shuangjianke(pais):
    """
    双箭刻, 牌里有 2 副箭刻（或杠）
    """
    return count_pais_by_types_and_range(data, ['kezi', 'gang'], 14, 16) == 2

def is_quandaiyao():
    """
    全带幺，胡牌时，每副牌、将牌都有幺牌。（胡牌时各组牌除了字牌都必须有一或九的序数牌）
    """
    pass

def is_BuQiuren(pais):
    """
    不求人，4 副牌及将中没有吃牌、碰牌（包括明杠），自摸胡牌。
    """
    pass

def isShuangminggang():
    """
    双明杠，牌里有 2 个明杠
    """
    
    for pai in data['gang'].values():
        if pai['zimo'] == 0:
            count += 1
            
    return  count == 2     

def is_tingpai():
    """
    听牌，玩家听牌则不可换牌，自动摸打
    """
    pass

def is_jianke(pais):
    """
    牌里有中、发、白，这3个牌中的任一个牌组成的 1 副刻子
    """
    return count_pais_by_types_and_range(data, ['kezi', 'gang'], 14, 16) == 1

def is_menqianqing(pais):
    """
    门前清，没有吃、碰、明杠，胡别人打出的牌
    """
    pass

def isSiguiyi():
    """
    四归一，牌里有 4 张相同的牌归于一家的顺、刻子、对、将牌（不包括杠牌）
    """
    pass

def isShuanganke():
    """
    双暗刻，牌里有2个暗刻
    """
    pass

def isAngang():
    """
    暗杠，牌里有一副自抓4张相同的牌且开杠
    """
    pass

def isDuan19(pais):
    """
    断幺九，牌里没有一、九牌及字牌
    """
    for pai in pais:
        if pai in [1, 9, 10, 11, 12, 13, 14, 15, 16]:
            return False
    return True

def isLian6(pais):
    """
    连六，一种花色六张序数相连的顺子
    """
    pass

def isLaoshaofu(pais):
    """
    老少一副，牌里花色相同的123、789的顺子各一副
    """
    pass

def isYaojiuke(pais):
    """
    幺九刻，牌里有序数为一、九的一副刻子（杠）或是字牌的一副刻子（杠）
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] >=3, pai_count.items()))) >= 1


def isMinggang(pais):
    """
    明杠, 自己有暗刻，碰别人打出的一张相同的牌开杠：或自己抓进一张与碰的明刻相同
    """
    pass

def isDanzhang(pais):
    """
    单张，单胡123的3及789的7或1233和3、7789和7都为边张。手中有12345或1112胡3，56789或8889胡7不算边张。
    """
    pass

def isKanzhang(pais):
    """
    坎张，看涨胡牌说明：胡牌时，胡2张牌之间的牌。4556和5也为坎张，手中有45567胡6不算坎张。不计边张、单调将。
    """
    pass

def isDandiaojiang(pais):
    """
    单钓将，看涨胡牌说明：钓单张牌作将成胡。不计边张坎张。
    """
    pass

def isZimo(pais):
    """
    自摸，自己摸的牌胡
    """
    pass

In [152]:
pais = np.array([1, 1, 10, 10, 10, 11, 11,11, 14,14, 14,6,6,6])
# isShibaluoghan(pais)


print(isDuan19(pais))

False


In [142]:
import json

text = [1, 2, 4]
text2 = [1]

print(text - text2)



TypeError: unsupported operand type(s) for -: 'list' and 'list'